# Finite-Difference Simulation of Lid-Driven Cavity Flow
**(Final Assignment of Computational Physics 2025)**



In [2]:
import numpy as np
import matplotlib.pyplot as plt

libertine_installed = True # Only set this to true if the font Linux Libertine is installed

In [3]:
plt.rcParams.update({
    "axes.prop_cycle": plt.cycler(color=["black"] 
                            + plt.rcParams['axes.prop_cycle'].by_key()['color']),
    "font.family": "serif",             # Serif fonts (e.g., Times-like)
    "font.size": 14,                    # Base font size
    "mathtext.fontset": "cm",           # Math Font to match a bit better
    "legend.fontsize": 12,
    "axes.labelsize": 14,
    "axes.titlesize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "figure.dpi": 300,                  # High resolution
    "savefig.dpi": 300,
    "axes.linewidth": 0.8,              # Thin axis lines
    "xtick.direction": "in",            # Ticks pointing inward
    "ytick.direction": "in",
    "xtick.top": True,                  # Ticks on all sides
    "ytick.right": True,
    "xtick.minor.visible": True,
    "ytick.minor.visible": True,
    "grid.linewidth": 0.4
})

if libertine_installed: plt.rcParams.update({"font.serif": ["Linux Libertine"]})  # Same font as in LaTeX)

fsize_small = (4,2.75) # for a single colum in double colum
fsize_large = (6.5,4)

## Building the Functions

ldc = LDC(h=A)
LDC.h = B
ldc.system_evolution()

In [4]:
class LDC:
    def __init__(self,h,dt,Re,n_t,grid_size,vx_wall):
        self.h = h
        self.dt = dt 
        self.Re = Re
        self.n_t = n_t
        self.grid_size = grid_size
        self.vx_wall = vx_wall
        self.initalize()
    
    def initialize(self): # Careful, for changing grid_size you have to reassign the whole class
        self.N_y, self.N_x = self.grid_size # first y then x, so if we print it out it corresponds to the figure

    def update_stream_SOR(self, u, w, omega, max_iter, tol):
        '''
        '''

        for it in range(max_iter):
            max_r = 0.0
            for j in range(1, self.N_y-1):
                for i in range(1, self.N_x-1):
                    u_new = 0.25 * (u[j+1,i] + u[j-1,i] + u[j,i+1] + u[j,i-1] + self.h*self.h * w[j,i])
                    r = u_new - u[j,i]
                    u[j,i] += omega * r
                    max_r = max(max_r, abs(r))

            if max_r < tol: break

        return u
    
    def update_vorticity(self,u, w):

        w_new = np.copy(w)

        for j in range(1, self.N_y-1):
            for i in range(1, self.N_x-1):
                term1a = (u[j,i+1] - u[j,i-1]) * (w[j+1,i] - w[j-1,i]) 
                term1b = (u[j+1,i] - u[j-1,i]) * (w[j,i+1] - w[j,i-1])
                term2 = w[j,i+1] + w[j,i-1] + w[j+1,i] + w[j-1,i] - 4*w[j,i]

                w_new[j,i] = w[j,i] + self.dt/(self.h*self.h) * (0.25 * (term1a - term1b) + (1/self.Re) * (term2))
        
        return w_new
    
    def system_evolution(self, 
                    SOR_omega=1.7, 
                    get_pressure=False, 
                    SOR_max_iter=10000, 
                    SOR_tol=1e-5
                    ):
        '''
        '''
        stability = self.dt/(self.Re*self.h*self.h)
        if stability > 0.25:
            print(f'Warning: Stability condition not met! value: {stability:.3f}')
        else:
            print(f'Stability criterion value: {stability:.3f}')

        # Initialize storage arrays and arrays at t=0
        u_history = np.zeros((self.n_t, self.N_y, self.N_x)) # u at t=0 is 0
        w_history = np.zeros((self.n_t, self.N_y, self.N_x))
        w_history[0,-1, :] = -2 * self.vx_wall / self.h  # Top wall vorticity at t=0 (u=0 everywhere)
                                                                        
        u = u_history[0]
        w = w_history[0]

        if get_pressure: 
            p_history = np.zeros((self.n_t, self.N_y, self.N_x))
            p = p_history[0]
                            
        for n in range(1,self.n_t):
            # Update stream function
            u = self.update_stream_SOR(u, w, self.h, omega=SOR_omega, max_iter=SOR_max_iter, tol=SOR_tol)

            # vorticity boundary conditions
            factor = 2/(self.h*self.h)
            w[-1, :] = - factor * u[-2,:] - (2/self.h) * self.vx_wall # Top wall (A)
            w[:, -1] = - factor * u[:,-2]             # Right wall (B)
            w[0, :] = - factor * u[1, :]            # Bottom wall (C)
            w[:, 0] = - factor * u[:, 1]            # Left wall (D)

            # Update vorticity
            w = self.update_vorticity(u, w, self.h, self.dt, self.Re)
            u_history[n] = u
            w_history[n] = w
            if get_pressure:
                p = np.zeros((self.N_y, self.N_x))  # Placeholder for pressure calculation
                p_history[n] = p

        if get_pressure:
            return u_history, w_history, p_history
        else:
            return u_history, w_history
            


Let's test the function for a small grid size and a low number of time steps:

In [5]:
ldc_test = LDC(h=0.1, dt=0.01, n_t=100, Re=20, grid_size=(20,20),vx_wall=1)
u_test, w_test = ldc_test.system_evolution()

# u_test, w_test = system_evolution(h=0.1, dt=0.01, n_t=100, Re=20, grid_size=(20,20),vx_wall=1)

plt.imshow(u_test[-1], origin='lower'); plt.title("Stream function")
plt.colorbar()
plt.figure()
plt.imshow(w_test[-1], origin='lower'); plt.title("Vorticity")
plt.colorbar()

AttributeError: 'LDC' object has no attribute 'initalize'